# Descripción del proyecto

La compañía Sweet Lift Taxi ha recopilado datos históricos sobre pedidos de taxis en los aeropuertos. Para atraer a más conductores durante las horas pico, necesitamos predecir la cantidad de pedidos de taxis para la próxima hora. Construye un modelo para dicha predicción.

La métrica RECM en el conjunto de prueba no debe ser superior a 48.

## Instrucciones del proyecto.

1. Descarga los datos y haz el remuestreo por una hora.
2. Analiza los datos
3. Entrena diferentes modelos con diferentes hiperparámetros. La muestra de prueba debe ser el 10% del conjunto de datos inicial.4. Prueba los datos usando la muestra de prueba y proporciona una conclusión.

## Descripción de los datos

Los datos se almacenan en el archivo `taxi.csv`. 	
El número de pedidos está en la columna `num_orders`.

## Preparación

In [29]:
import pandas as pd
import numpy as np

import plotly.graph_objects as go
import plotly_express as px
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt

from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb
from catboost import CatBoostRegressor



In [30]:
data = pd.read_csv("https://practicum-content.s3.us-west-1.amazonaws.com/datasets/taxi.csv?etag=11687de0e23962e5a11c9d8ae13eb630", index_col=0, parse_dates=[0])

In [31]:
data

,num_orders
datetime,
2018-03-01 00:00:00,9
2018-03-01 00:10:00,14
2018-03-01 00:20:00,28
2018-03-01 00:30:00,20
2018-03-01 00:40:00,32
...,...
2018-08-31 23:10:00,32
2018-08-31 23:20:00,24
2018-08-31 23:30:00,27


## Análisis

In [32]:
data.isna().sum()
np.random.seed(999)

In [33]:
data_day = data.resample("1D").sum()
data_day.head()

,num_orders
datetime,
2018-03-01,1364
2018-03-02,1506
2018-03-03,1290
2018-03-04,1141
2018-03-05,1259


In [34]:
decomposed = seasonal_decompose(data_day['num_orders'], model='additive')

fig = make_subplots(rows=4, cols=1, subplot_titles=("Original","Tendencia", "Estacionalidad", "Residuales"), vertical_spacing= 0.07)

for i, observation in enumerate(["observed", "trend", "seasonal", "resid"], start=1):
    dataframe = pd.DataFrame({'ds': data_day.index, 'observation': getattr(decomposed, observation)})
    fig.add_trace(go.Scatter(x=dataframe['ds'], y=dataframe['observation'],showlegend=False), row=i, col=1)

fig.update_layout(height=1000, width=1000, title_text="Descomposición de Serie Temporal")
fig.show()

## Formación

In [35]:
def make_features(data, max_lag, rolling_mean_size):
    data['year'] = data.index.year
    data['month'] = data.index.month
    data['day'] = data.index.day
    data['dayofweek'] = data.index.dayofweek
    
    for lag in range(1, max_lag + 1):
        data['lag_{}'.format(lag)] = data['num_orders'].shift(lag)

    data["rolling_mean"] = data['num_orders'].shift().rolling(rolling_mean_size).mean()

make_features(data, 4, 4)
display(data.head(15))

,num_orders,year,month,day,dayofweek,lag_1,lag_2,lag_3,lag_4,rolling_mean
datetime,,,,,,,,,,
2018-03-01 00:00:00,9,2018,3,1,3,NaN,NaN,NaN,NaN,NaN
2018-03-01 00:10:00,14,2018,3,1,3,9.0,NaN,NaN,NaN,NaN
2018-03-01 00:20:00,28,2018,3,1,3,14.0,9.0,NaN,NaN,NaN
2018-03-01 00:30:00,20,2018,3,1,3,28.0,14.0,9.0,NaN,NaN
2018-03-01 00:40:00,32,2018,3,1,3,20.0,28.0,14.0,9.0,17.75
2018-03-01 00:50:00,21,2018,3,1,3,32.0,20.0,28.0,14.0,23.50
2018-03-01 01:00:00,7,2018,3,1,3,21.0,32.0,20.0,28.0,25.25
2018-03-01 01:10:00,5,2018,3,1,3,7.0,21.0,32.0,20.0,20.00
2018-03-01 01:20:00,17,2018,3,1,3,5.0,7.0,21.0,32.0,16.25


## Prueba

In [36]:
data.dropna(inplace=True)
X = data.drop("num_orders", axis=1)
y = data["num_orders"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, shuffle= False)


#### Regresión lineal 

In [37]:
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

predictions_train = lr_model.predict(X_train)
predictions_test = lr_model.predict(X_test)

print("rmse train:",root_mean_squared_error(y_train, predictions_train))
print("rmse test:",root_mean_squared_error(y_test, predictions_test))

rmse train: 5.426587050425898
rmse test: 7.806230049188653


#### Bosque Aleatorio


In [38]:

rf_model = RandomForestRegressor(max_features='sqrt', max_depth=10)

rf_model.fit(X_train, y_train)
rf_predictions = rf_model.predict(X_test)

rmse = root_mean_squared_error(y_test, rf_predictions)
print(f'RMSE: {rmse:.2f}')

# feature_importances = rf_model.feature_importances_
# features = X_train.columns
# indices = np.argsort(feature_importances)[-10:]


# plt.figure(figsize=(10, 8))
# plt.title('Importancia de las Características')
# plt.barh(range(len(indices)), feature_importances[indices], color='blue', align='center')
# plt.yticks(range(len(indices)), [features[i] for i in indices])
# plt.xlabel('Importancia Relativa')
# plt.show()

RMSE: 8.66


#### Bosque Aleatorio


In [45]:

lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test,y_test)

lgb_params = {
    "boosting_type": "gbdt",
    "objective": "regression",
    "metric": {"l2", "l1"},
    "num_leaves": 31,
    "learning_rate": 0.05,
    "feature_fraction": 0.9,
    "bagging_fraction": 0.8,
    "bagging_freq": 5,
    "verbose": 0,
}

lgb_model = lgb.train(lgb_params, lgb_train, valid_sets=lgb_eval)

lgb_predictions = lgb_model.predict(X_test, num_iteration=lgb_model.best_iteration)


rmse = root_mean_squared_error(y_test, lgb_predictions)
print(f'RMSE: {rmse:.2f}')


RMSE: 8.48


#### CatBoost Regressor

In [52]:
cat_model = CatBoostRegressor(
    iterations=1000,
    learning_rate=0.03,
    depth=6,
    loss_function='RMSE',
    verbose=False
)

cat_model.fit(X_train, y_train)
cat_predictions = cat_model.predict(X_test)

rmse = root_mean_squared_error(y_test, cat_predictions)
print(f'RMSE: {rmse:.2f}')

RMSE: 8.68


# Lista de revisión

- [x]  	
Jupyter Notebook está abierto.
- [ ]  El código no tiene errores
- [ ]  Las celdas con el código han sido colocadas en el orden de ejecución.
- [ ]  	
Los datos han sido descargados y preparados.
- [ ]  Se ha realizado el paso 2: los datos han sido analizados
- [ ]  Se entrenó el modelo y se seleccionaron los hiperparámetros
- [ ]  Se han evaluado los modelos. Se expuso una conclusión
- [ ] La *RECM* para el conjunto de prueba no es más de 48